# Orderbook 기초 - Intraday Trading 입문

이 노트북에서는 Binance의 실시간 Orderbook 데이터를 수신하고 분석하는 방법을 배웁니다.

## 학습 목표

1. **Orderbook (호가창)** 이해하기
2. **Bid-Ask Spread** 계산하고 해석하기
3. **Mid-price vs Micro-price** 차이 이해하기
4. **Orderbook 히트맵** 시각화하기

---

## 1. 환경 설정

In [3]:
import asyncio
import sys
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



# 프로젝트 모듈 import
sys.path.insert(0, '../src')
from intraday.client import BinanceWebSocketClient, OrderbookSnapshot
from intraday.orderbook import OrderbookProcessor, OrderbookState
from intraday.metrics import MetricsCalculator, MetricsSnapshot

print("모듈 로드 완료!")

모듈 로드 완료!


---

## 2. Orderbook (호가창) 이해하기

### 개념 설명

**Orderbook**은 특정 자산에 대한 매수(Bid)와 매도(Ask) 주문을 가격별로 정렬한 목록입니다.

```
           ORDERBOOK (호가창)              
    매수 (Bid)    |      매도 (Ask)         
   "사고 싶다"     |     "팔고 싶다"          
--------------------------------------------
 $99,950 (2.5)  |  $100,050 (1.2)  <- Best Ask
 $99,900 (1.8)  |  $100,100 (3.4)        
 $99,850 (4.2)  |  $100,150 (2.1)        
      ^         |                       
 Best Bid       |                        
```

**핵심 포인트:**
- **Best Bid**: 가장 높은 매수 가격 (시장가 매도 시 체결 가격)
- **Best Ask**: 가장 낮은 매도 가격 (시장가 매수 시 체결 가격)
- **Spread**: Best Ask - Best Bid (거래 비용)

### 실습: Binance에서 실시간 데이터 수신

Binance WebSocket을 통해 BTC/USDT Orderbook 데이터를 수신해봅시다.

In [2]:
# 데이터 수집기 초기화
processor = OrderbookProcessor(max_history=500)
metrics_calc = MetricsCalculator(max_history=500)
collected_data = []

async def collect_orderbook_data(num_samples: int = 100):
    """
    Binance에서 Orderbook 데이터를 수집합니다.
    
    Args:
        num_samples: 수집할 샘플 수 (100ms 간격이므로 100개 = 약 10초)
    """
    client = BinanceWebSocketClient("btcusdt", depth_levels=20, update_speed="100ms")
    count = 0
    
    def on_data(snapshot: OrderbookSnapshot):
        nonlocal count
        count += 1
        
        # 데이터 처리
        state = processor.update(snapshot)
        metrics = metrics_calc.calculate(state)
        collected_data.append(metrics)
        
        # 진행 상황 출력
        if count % 20 == 0:
            print(f"수집 중... {count}/{num_samples} | "
                  f"Mid: ${state.mid_price:,.2f} | "
                  f"Spread: ${state.spread:.2f}")
        
        # 목표 샘플 수 도달 시 종료
        if count >= num_samples:
            asyncio.create_task(client.disconnect())
    
    print(f"BTC/USDT Orderbook 데이터 {num_samples}개 수집 시작...")
    print(f"   (약 {num_samples * 0.1:.1f}초 소요 예상)\n")
    
    await client.connect(on_data)
    
    print(f"\n데이터 수집 완료! {len(collected_data)}개 샘플")

# 데이터 수집 실행 (약 10초)
await collect_orderbook_data(100)

BTC/USDT Orderbook 데이터 100개 수집 시작...
   (약 10.0초 소요 예상)

[Client] Connecting to wss://stream.binance.com:9443/ws/btcusdt@depth20@100ms...
[Client] Connected! Receiving BTCUSDT orderbook...
수집 중... 20/100 | Mid: $87,555.51 | Spread: $0.01
수집 중... 40/100 | Mid: $87,565.46 | Spread: $0.01
수집 중... 60/100 | Mid: $87,577.99 | Spread: $0.01
수집 중... 80/100 | Mid: $87,577.99 | Spread: $0.01
수집 중... 100/100 | Mid: $87,579.99 | Spread: $0.01
[Client] WebSocket close timed out, socket might be left open.
[Client] Disconnected.

데이터 수집 완료! 100개 샘플


In [3]:
# 수집된 데이터 확인
df = metrics_calc.to_dataframe()
print(f"수집된 데이터: {len(df)} rows\n")
print("처음 5개 행:")
df.head()

수집된 데이터: 100 rows

처음 5개 행:


,symbol,best_bid,best_ask,mid_price,micro_price,spread,spread_bps,bid_qty,ask_qty,imbalance,mid_price_change,micro_price_change
timestamp,,,,,,,,,,,,
2025-12-23 17:05:59.673670,BTCUSDT,87555.5,87555.51,87555.505,87555.508759,0.01,0.001142,3.24269,0.45952,0.751759,0.0,0.000000e+00
2025-12-23 17:05:59.773661,BTCUSDT,87555.5,87555.51,87555.505,87555.508759,0.01,0.001142,3.24269,0.45958,0.751731,0.0,-1.419394e-07
2025-12-23 17:05:59.874085,BTCUSDT,87555.5,87555.51,87555.505,87555.508759,0.01,0.001142,3.24270,0.45958,0.751731,0.0,3.346941e-09
2025-12-23 17:05:59.973836,BTCUSDT,87555.5,87555.51,87555.505,87555.508759,0.01,0.001142,3.24270,0.45958,0.751731,0.0,0.000000e+00
2025-12-23 17:06:00.073844,BTCUSDT,87555.5,87555.51,87555.505,87555.508759,0.01,0.001142,3.24270,0.45958,0.751731,0.0,0.000000e+00


In [4]:
# 기술 통계
print("기술 통계:")
df[['mid_price', 'micro_price', 'spread', 'spread_bps', 'imbalance']].describe()

기술 통계:


,mid_price,micro_price,spread,spread_bps,imbalance
count,100.000000,100.000000,1.000000e+02,1.000000e+02,100.000000
mean,87568.276600,87568.280061,1.000000e-02,1.141966e-03,0.692139
std,10.016112,10.015442,1.455192e-12,1.306208e-07,0.327692
min,87555.505000,87555.508567,1.000000e-02,1.141813e-03,-0.009351
25%,87555.505000,87555.509922,1.000000e-02,1.141839e-03,0.661161
50%,87565.465000,87565.469997,1.000000e-02,1.142003e-03,0.752399
75%,87577.995000,87577.997348,1.000000e-02,1.142133e-03,0.982468
max,87579.995000,87579.999913,1.000000e-02,1.142133e-03,0.999431


---

## 3. Bid-Ask Spread 분석

### 개념 설명

**Spread (스프레드)** = Best Ask - Best Bid

스프레드는 다음을 의미합니다:

1. **거래 비용**: 즉시 사고팔면 스프레드만큼 손해
2. **유동성 지표**: 스프레드가 좁을수록 유동성 높음
3. **시장 효율성**: 효율적인 시장일수록 스프레드가 좁음

**Basis Points (bps)**: 상대적 스프레드
- 1 bps = 0.01% = 0.0001
- BTC/USDT: 보통 0.1~2 bps (매우 유동적)
- 알트코인: 5~50+ bps (유동성 낮음)

In [5]:
# Spread 시계열 시각화
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Spread ($)', 'Spread (bps)'),
    vertical_spacing=0.12
)

# 절대 스프레드
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['spread'],
        mode='lines',
        name='Spread ($)',
        line=dict(color='#00D4AA', width=1.5)
    ),
    row=1, col=1
)

# 상대 스프레드 (bps)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['spread_bps'],
        mode='lines',
        name='Spread (bps)',
        line=dict(color='#FF6B6B', width=1.5)
    ),
    row=2, col=1
)

fig.update_layout(
    title='Bid-Ask Spread 시계열',
    height=500,
    template='plotly_dark',
    showlegend=True
)

fig.update_yaxes(title_text='$', row=1, col=1)
fig.update_yaxes(title_text='bps', row=2, col=1)

fig.show()

In [6]:
# Spread 분포
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['spread'],
    nbinsx=100,
    name='Spread Distribution',
    marker_color='#00D4AA'
))

# 평균선 추가
mean_spread = df['spread'].mean()
fig.add_vline(
    x=mean_spread,
    line_dash="dash",
    line_color="yellow",
    annotation_text=f"평균: ${mean_spread:.2f}"
)

fig.update_layout(
    title='Spread 분포',
    xaxis_title='Spread ($)',
    yaxis_title='빈도',
    template='plotly_dark',
    height=400
)

fig.show()

---

## 4. Mid-price vs Micro-price

### 개념 설명

**Mid-price (중간가)**
```
Mid-price = (Best Bid + Best Ask) / 2
```
- 단순 산술 평균
- 주문량을 고려하지 않음

**Micro-price (마이크로 프라이스)**
```
Micro-price = (Best Bid * Ask Qty + Best Ask * Bid Qty) / (Bid Qty + Ask Qty)
```
- 주문량으로 가중한 평균
- 주문 불균형을 반영

### 예시

```
Best Bid: $100 (10 BTC 대기)
Best Ask: $101 (1 BTC 대기)

Mid-price = ($100 + $101) / 2 = $100.50

Micro-price = ($100 * 1 + $101 * 10) / (10 + 1)
            = ($100 + $1010) / 11
            = $100.91
```

**해석**: 매도 물량(1 BTC)이 매수 물량(10 BTC)보다 적으므로, 
Micro-price가 Mid-price보다 높습니다 -> **가격 상승 압력**

In [7]:
# Mid-price vs Micro-price 비교
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Mid-price vs Micro-price', 'Micro - Mid Difference'),
    vertical_spacing=0.12
)

# 가격 비교
fig.add_trace(
    go.Scatter(
        x=df.index, y=df['mid_price'],
        mode='lines', name='Mid-price',
        line=dict(color='#4ECDC4', width=1.5)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index, y=df['micro_price'],
        mode='lines', name='Micro-price',
        line=dict(color='#FF6B6B', width=1.5)
    ),
    row=1, col=1
)

# 차이 (Micro - Mid)
diff = df['micro_price'] - df['mid_price']
colors = ['#00D4AA' if d >= 0 else '#FF6B6B' for d in diff]

fig.add_trace(
    go.Bar(
        x=df.index, y=diff,
        name='Micro - Mid',
        marker_color=colors
    ),
    row=2, col=1
)

fig.add_hline(y=0, line_dash="dash", line_color="white", row=2, col=1)

fig.update_layout(
    title='Mid-price vs Micro-price 비교',
    height=600,
    template='plotly_dark',
    showlegend=True
)

fig.update_yaxes(title_text='Price ($)', row=1, col=1)
fig.update_yaxes(title_text='Difference ($)', row=2, col=1)

fig.show()

In [8]:
# 통계 요약
diff = df['micro_price'] - df['mid_price']

print("Mid-price vs Micro-price 분석\n")
print(f"평균 Mid-price:   ${df['mid_price'].mean():,.2f}")
print(f"평균 Micro-price: ${df['micro_price'].mean():,.2f}")
print(f"\n평균 차이 (Micro - Mid): ${diff.mean():.4f}")
print(f"표준편차:                 ${diff.std():.4f}")
print(f"\nMicro > Mid 비율: {(diff > 0).mean() * 100:.1f}%")
print(f"Micro < Mid 비율: {(diff < 0).mean() * 100:.1f}%")

if diff.mean() > 0:
    print("\n해석: Micro-price가 평균적으로 높음 -> 매도 물량 < 매수 물량 -> 상승 압력")
else:
    print("\n해석: Micro-price가 평균적으로 낮음 -> 매도 물량 > 매수 물량 -> 하락 압력")

Mid-price vs Micro-price 분석

평균 Mid-price:   $87,568.28
평균 Micro-price: $87,568.28

평균 차이 (Micro - Mid): $0.0035
표준편차:                 $0.0016

Micro > Mid 비율: 98.0%
Micro < Mid 비율: 2.0%

해석: Micro-price가 평균적으로 높음 -> 매도 물량 < 매수 물량 -> 상승 압력


---

## 5. Orderbook 히트맵 시각화

### 개념 설명

**Orderbook 히트맵**은 가격별 주문량을 색상 강도로 표현합니다.

- **진한 색상**: 큰 주문량 ("벽"이라고 부름)
- **연한 색상**: 작은 주문량

트레이더들은 히트맵을 통해:
1. 지지/저항 수준 파악
2. 큰 주문(고래)의 위치 확인
3. 주문 취소/추가 패턴 관찰

In [9]:
# 현재 Orderbook 스냅샷 시각화
current_state = processor.current

if current_state:
    # 데이터 준비
    bid_prices = current_state.bid_prices[:15]
    bid_qtys = current_state.bid_quantities[:15]
    ask_prices = current_state.ask_prices[:15]
    ask_qtys = current_state.ask_quantities[:15]
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('매수 (Bids)', '매도 (Asks)'),
        horizontal_spacing=0.1
    )
    
    # 매수 호가 (가로 막대)
    fig.add_trace(
        go.Bar(
            y=[f"${p:,.0f}" for p in bid_prices],
            x=bid_qtys,
            orientation='h',
            name='Bids',
            marker_color='#00D4AA',
            text=[f"{q:.3f}" for q in bid_qtys],
            textposition='outside'
        ),
        row=1, col=1
    )
    
    # 매도 호가 (가로 막대)
    fig.add_trace(
        go.Bar(
            y=[f"${p:,.0f}" for p in ask_prices],
            x=ask_qtys,
            orientation='h',
            name='Asks',
            marker_color='#FF6B6B',
            text=[f"{q:.3f}" for q in ask_qtys],
            textposition='outside'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title=f'BTC/USDT Orderbook (상위 15개 호가)<br>'
              f'<sup>Best Bid: ${current_state.best_bid[0]:,.2f} | '
              f'Best Ask: ${current_state.best_ask[0]:,.2f} | '
              f'Spread: ${current_state.spread:.2f}</sup>',
        height=600,
        template='plotly_dark',
        showlegend=False
    )
    
    fig.update_xaxes(title_text='Quantity (BTC)', row=1, col=1)
    fig.update_xaxes(title_text='Quantity (BTC)', row=1, col=2)
    
    fig.show()
else:
    print("데이터가 없습니다. 먼저 데이터를 수집하세요.")

In [10]:
# Depth Chart (누적 주문량)
if current_state:
    depth_data = processor.get_depth_chart_data(num_levels=20)
    
    fig = go.Figure()
    
    # 매수 누적 (왼쪽에서 오른쪽으로)
    fig.add_trace(go.Scatter(
        x=depth_data['bid_prices'][::-1],
        y=depth_data['bid_cumulative'][::-1],
        fill='tozeroy',
        name='Bids (누적 매수)',
        line=dict(color='#00D4AA'),
        fillcolor='rgba(0, 212, 170, 0.3)'
    ))
    
    # 매도 누적 (왼쪽에서 오른쪽으로)
    fig.add_trace(go.Scatter(
        x=depth_data['ask_prices'],
        y=depth_data['ask_cumulative'],
        fill='tozeroy',
        name='Asks (누적 매도)',
        line=dict(color='#FF6B6B'),
        fillcolor='rgba(255, 107, 107, 0.3)'
    ))
    
    # Mid-price 표시
    fig.add_vline(
        x=current_state.mid_price,
        line_dash="dash",
        line_color="yellow",
        annotation_text=f"Mid: ${current_state.mid_price:,.2f}"
    )
    
    fig.update_layout(
        title='Depth Chart (누적 주문량)',
        xaxis_title='Price ($)',
        yaxis_title='Cumulative Quantity (BTC)',
        template='plotly_dark',
        height=500
    )
    
    fig.show()

---

## 6. Order Imbalance (주문 불균형)

### 개념 설명

**Order Imbalance** = (Bid Qty - Ask Qty) / (Bid Qty + Ask Qty)

- 범위: -1 ~ +1
- **+1에 가까움**: 매수 물량 >> 매도 물량 (상승 압력)
- **-1에 가까움**: 매도 물량 >> 매수 물량 (하락 압력)
- **0 근처**: 균형 상태

In [11]:
# Imbalance 시계열
fig = go.Figure()

# 색상: 양수는 초록, 음수는 빨강
colors = ['#00D4AA' if i >= 0 else '#FF6B6B' for i in df['imbalance']]

fig.add_trace(go.Bar(
    x=df.index,
    y=df['imbalance'],
    marker_color=colors,
    name='Order Imbalance'
))

# 기준선
fig.add_hline(y=0, line_dash="solid", line_color="white", line_width=1)
fig.add_hline(y=0.3, line_dash="dash", line_color="#00D4AA", 
              annotation_text="Strong Buy (+0.3)")
fig.add_hline(y=-0.3, line_dash="dash", line_color="#FF6B6B",
              annotation_text="Strong Sell (-0.3)")

fig.update_layout(
    title='Order Imbalance 시계열',
    xaxis_title='Time',
    yaxis_title='Imbalance',
    yaxis=dict(range=[-1, 1]),
    template='plotly_dark',
    height=400
)

fig.show()

In [12]:
# Imbalance vs 가격 변화 상관관계
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['imbalance'],
    y=df['mid_price_change'],
    mode='markers',
    marker=dict(
        color=df['imbalance'],
        colorscale='RdYlGn',
        size=8,
        opacity=0.6
    ),
    name='Imbalance vs Price Change'
))

# 추세선 (선형 회귀)
z = np.polyfit(df['imbalance'].dropna(), df['mid_price_change'].dropna(), 1)
p = np.poly1d(z)
x_line = np.linspace(df['imbalance'].min(), df['imbalance'].max(), 100)

fig.add_trace(go.Scatter(
    x=x_line,
    y=p(x_line),
    mode='lines',
    line=dict(color='yellow', dash='dash'),
    name='Trend'
))

fig.update_layout(
    title='Order Imbalance vs Price Change',
    xaxis_title='Order Imbalance',
    yaxis_title='Mid-price Change ($)',
    template='plotly_dark',
    height=500
)

fig.show()

# 상관계수
corr = df['imbalance'].corr(df['mid_price_change'])
print(f"\n상관계수: {corr:.4f}")
if corr > 0.1:
    print("양의 상관관계: Imbalance가 높을수록 가격이 오르는 경향")
elif corr < -0.1:
    print("음의 상관관계: Imbalance가 높을수록 가격이 내리는 경향")
else:
    print("약한 상관관계: 단기적으로 명확한 패턴 없음")


상관계수: 0.2118
양의 상관관계: Imbalance가 높을수록 가격이 오르는 경향


---

## 📚 전체 요약 및 핵심 포인트

### 학습한 내용

#### 1. **Orderbook (호가창)**
   - 매수(Bid)와 매도(Ask) 주문이 가격별로 정렬된 데이터
   - Best Bid = 최고 매수가, Best Ask = 최저 매도가
   - **의도**를 보여줌 (아직 체결되지 않은 주문)

#### 2. **Bid-Ask Spread**
   - Spread = Best Ask - Best Bid
   - 유동성 지표 (좁을수록 유동성 높음)
   - BTC/USDT는 매우 좁은 스프레드 (높은 유동성)

#### 3. **Mid-price vs Micro-price**
   - Mid-price: 단순 중간가
   - Micro-price: 주문량 가중 중간가
   - 차이 > 0: 상승 압력, 차이 < 0: 하락 압력

#### 4. **Order Imbalance**
   - (Bid Qty - Ask Qty) / Total
   - 주문 불균형을 수치화

#### 5. **Footprint Chart (발자국 차트)** ⭐ NEW
   - 각 가격/시간에서 실제 체결된 거래량을 매수/매도로 구분
   - **결과**를 보여줌 (실제로 체결된 거래)
   - Delta = 매수량 - 매도량
   - Cumulative Delta로 추세의 강도 파악

### 핵심 비교: Orderbook vs Footprint

| 구분 | Orderbook (호가창) | Footprint (발자국) |
|------|-------------------|-------------------|
| **의미** | 의도 (주문) | 결과 (체결) |
| **데이터** | 대기 중인 주문 | 실제 거래 |
| **지표** | Imbalance, Spread | Delta, Volume |
| **활용** | 단기 압력 파악 | 추세 확인, 지지/저항 |

### 실전 활용 전략

1. **Orderbook으로 단기 압력 확인**
   - Imbalance > 0.3 → 매수 압력 강함
   - Imbalance < -0.3 → 매도 압력 강함

2. **Footprint로 추세 확인**
   - Cumulative Delta 상승 → 강세 추세
   - Cumulative Delta 하락 → 약세 추세

3. **두 지표를 함께 사용**
   - Orderbook 매수 압력 + Footprint 매수 체결 → 강한 상승 신호
   - Orderbook 매도 압력 + Footprint 매도 체결 → 강한 하락 신호
   - 불일치 발생 → 주의 (페이크 신호 가능)

### 다음 단계

실시간 대시보드에서 Orderbook과 Footprint를 동시에 모니터링해보세요!

```bash
# 실시간 대시보드 실행
uv run python -m intraday.dashboard
```

**🎓 학습 완료!** 이제 Intraday Trading의 기초를 이해했습니다. 다음 노트북에서는 더 고급 전략을 배워봅시다.

---

## 5. Footprint Chart (발자국 차트)

### 개념 설명

**Footprint Chart**는 각 가격 레벨에서 발생한 실제 거래량을 매수/매도로 구분하여 시각화한 차트입니다.

```
     시간 →
가격 ↑  [캔들1]  [캔들2]  [캔들3]
$100  매수:50  매수:30  매수:20
      매도:10  매도:40  매도:60
      
$99   매수:80  매수:90  매수:40
      매도:20  매도:15  매도:70
```

**핵심 포인트:**
- **매수량 > 매도량**: 해당 가격에서 매수 압력이 강함 (초록색)
- **매도량 > 매수량**: 해당 가격에서 매도 압력이 강함 (빨간색)
- **Delta**: 매수량 - 매도량 (거래 흐름의 방향성)

### 왜 중요한가?

1. **주문 흐름 파악**: 어느 가격대에서 실제로 거래가 많이 일어났는지
2. **지지/저항 레벨 발견**: 특정 가격대에서 거래량이 몰리는 곳
3. **Orderbook 데이터와 보완**: 호가창은 의도(주문), Footprint은 실제(체결)

In [ ]:
### 거래 데이터 로드 (Binance 히스토리컬 데이터)

from pathlib import Path
from datetime import datetime
from intraday.client import AggTrade
from intraday.data import TickDataDownloader, TickDataLoader

# 설정
data_dir = Path("../data/ticks")
symbol = "BTCUSDT"
target_date = datetime(2024, 12, 20)  # 원하는 날짜로 변경
max_trades = 10000  # 로드할 최대 거래 수

# 다운로드 (파일이 있으면 스킵)
downloader = TickDataDownloader()
filepath = downloader.download_daily(symbol=symbol, date=target_date, output_dir=data_dir)

# 로드
loader = TickDataLoader(filepath)
trades_data: list[AggTrade] = []

for i, trade in enumerate(loader.iter_trades()):
    trades_data.append(trade)
    if i + 1 >= max_trades:
        break

print(f"✅ 로드 완료: {len(trades_data):,}개")
print(f"   기간: {trades_data[0].timestamp} ~ {trades_data[-1].timestamp}")

[Downloader] Downloading https://data.binance.vision/data/spot/daily/aggTrades/BTCUSDT/BTCUSDT-aggTrades-2024-12-20.zip...
[Downloader] Downloaded 40330.3 KB
[Downloader] Saved to ../data/ticks/BTCUSDT-aggTrades-2024-12-20.parquet (3,004,257 records)
✅ 데이터 다운로드 완료: ../data/ticks/BTCUSDT-aggTrades-2024-12-20.parquet
[TickDataLoader] Found 1 file(s)

데이터 로드 중... (최대 10,000개)
  로드 중... 2,000/10,000
  로드 중... 4,000/10,000
  로드 중... 6,000/10,000
  로드 중... 8,000/10,000

✅ 거래 데이터 로드 완료! 10,000개
   기간: 2024-12-20 00:00:00.289000 ~ 2024-12-20 00:05:18.005000
   가격 범위: $97,461.86 ~ $97,516.89


In [14]:
# 거래 데이터를 DataFrame으로 변환
# AggTrade.is_buyer_maker의 의미:
# - True: Buyer가 Maker (Ask에 지정가 주문) → Seller가 시장가로 체결 → Seller가 Taker → 왼쪽 (Sell)
# - False: Seller가 Maker (Bid에 지정가 주문) → Buyer가 시장가로 체결 → Buyer가 Taker → 오른쪽 (Buy)
#
# Footprint Chart 해석:
# - 왼쪽 (Sell): "Sellers who were hitting the bid" = Seller가 Bid를 hit (시장가 매도)
# - 오른쪽 (Buy): "Buyers who lifted the order" = Buyer가 Ask를 lift (시장가 매수)
trades_df = pd.DataFrame([
    {
        'timestamp': t.timestamp,
        'price': t.price,
        'quantity': t.quantity,
        'side': 'sell' if t.is_buyer_maker else 'buy'  # is_buyer_maker=True → Seller가 Taker → Sell
    }
    for t in trades_data
])

print(f"총 {len(trades_df)}개의 거래 데이터")
print(f"\n거래 방향별 통계:")
side_stats = trades_df.groupby('side').agg({
    'quantity': ['count', 'sum', 'mean']
}).round(4)
print(side_stats)

수집된 거래: 10000 rows

                timestamp     price  quantity  side
0 2024-12-20 00:00:00.289  97461.86   0.00373  sell
1 2024-12-20 00:00:00.298  97461.86   0.11381  sell
2 2024-12-20 00:00:00.405  97461.86   0.02728  sell
3 2024-12-20 00:00:00.514  97460.00   0.00008   buy
4 2024-12-20 00:00:00.514  97460.00   0.04730  sell

거래 방향별 통계:
     quantity                 
        count      sum    mean
side                          
buy      5285  67.0193  0.0127
sell     4715  76.4261  0.0162


In [15]:
### Footprint 데이터 가공

# ============================================================================
# Step 1: 시간을 캔들로 그룹화 (예: 몇초 봉이냐)
# ============================================================================
# trades_df 샘플:
#   timestamp            candle_time  price    quantity  side
#   2024-01-01 10:00:03  10:00:00     97342.51  0.5      buy
#   2024-01-01 10:00:07  10:00:05     97345.23  1.2      sell
candle_duration = pd.Timedelta(seconds=30)
trades_df['candle_time'] = trades_df['timestamp'].dt.floor(candle_duration)

# ============================================================================
# Step 2: 가격을 적절한 레벨로 그룹화 (예: $1 단위)
# ============================================================================
# BTC는 가격이 높으므로 $5-10 정도 간격이 적절
# trades_df 샘플:
#   timestamp            candle_time  price    price_level  quantity  side
#   2024-01-01 10:00:03  10:00:00     97342.51  97340.0      0.5      buy
#   2024-01-01 10:00:07  10:00:05     97345.23  97345.0      1.2      sell
price_tick = 5.0
trades_df['price_level'] = (trades_df['price'] / price_tick).round() * price_tick

# ============================================================================
# Step 3: 캔들별, 가격별, 거래방향별 거래량 집계
# ============================================================================
# groupby(['candle_time', 'price_level', 'side'])는 세 컬럼의 조합으로 그룹화합니다.
# 예: (10:00:00, 97340.0, 'buy') 그룹, (10:00:00, 97340.0, 'sell') 그룹 등
# 각 조합별로 quantity를 합산합니다.
#
# footprint_data 샘플:
#   candle_time  price_level  side  quantity
#   10:00:00     97340.0      buy   1.2      <- (10:00:00, 97340.0, 'buy') 그룹의 합
#   10:00:00     97340.0      sell  0.5      <- (10:00:00, 97340.0, 'sell') 그룹의 합
#   10:00:05     97345.0      buy   0.8      <- (10:00:05, 97345.0, 'buy') 그룹의 합
footprint_data = trades_df.groupby(
    ['candle_time', 'price_level', 'side']
)['quantity'].sum().reset_index()

# ============================================================================
# Step 4: Pivot하여 매수/매도를 분리
# ============================================================================
# footprint_pivot 샘플:
#   price_level  (10:00:00, 'buy')  (10:00:00, 'sell')  (10:00:05, 'buy')  ...
#   97340.0      1.2                0.5                 0.0                ...
#   97345.0      0.0                0.0                 0.8                ...
footprint_pivot = footprint_data.pivot_table(
    index='price_level',
    columns=['candle_time', 'side'],
    values='quantity',
    fill_value=0
)

# ============================================================================
# Step 5: Delta 계산 (매수 - 매도)
# ============================================================================
# footprint_delta 샘플:
#   {
#       (10:00:00, 97340.0): {'buy': 1.2, 'sell': 0.5, 'delta': 0.7, 'total': 1.7},
#       (10:00:05, 97345.0): {'buy': 0.8, 'sell': 0.3, 'delta': 0.5, 'total': 1.1},
#       ...
#   }
footprint_delta = {}
for candle in trades_df['candle_time'].unique():
    for price in trades_df['price_level'].unique():
        try:
            buy_vol = footprint_pivot.loc[price, (candle, 'buy')]
        except:
            buy_vol = 0
        try:
            sell_vol = footprint_pivot.loc[price, (candle, 'sell')]
        except:
            sell_vol = 0
        
        footprint_delta[(candle, price)] = {
            'buy': buy_vol,
            'sell': sell_vol,
            'delta': buy_vol - sell_vol,
            'total': buy_vol + sell_vol
        }
print(footprint_delta)

{(Timestamp('2024-12-20 00:00:00'), np.float64(97460.0)): {'buy': np.float64(6.3181), 'sell': np.float64(2.42088), 'delta': np.float64(3.8972200000000004), 'total': np.float64(8.73898)}, (Timestamp('2024-12-20 00:00:00'), np.float64(97465.0)): {'buy': np.float64(0.055639999999999995), 'sell': np.float64(0.0), 'delta': np.float64(0.055639999999999995), 'total': np.float64(0.055639999999999995)}, (Timestamp('2024-12-20 00:00:00'), np.float64(97470.0)): {'buy': np.float64(0.26216), 'sell': np.float64(0.7091999999999999), 'delta': np.float64(-0.44703999999999994), 'total': np.float64(0.97136)}, (Timestamp('2024-12-20 00:00:00'), np.float64(97475.0)): {'buy': np.float64(0.15606), 'sell': np.float64(0.64425), 'delta': np.float64(-0.48819), 'total': np.float64(0.80031)}, (Timestamp('2024-12-20 00:00:00'), np.float64(97480.0)): {'buy': np.float64(0.05964), 'sell': np.float64(0.0), 'delta': np.float64(0.05964), 'total': np.float64(0.05964)}, (Timestamp('2024-12-20 00:00:00'), np.float64(97485.0

In [16]:
### Footprint Chart 시각화

# ============================================================================
# Step 1: 고유한 캔들 시간과 가격 레벨 추출
# ============================================================================
# trades_df 샘플:
#   timestamp            candle_time  price    price_level  quantity  side
#   2024-01-01 10:00:03  10:00:00     97342.51  97340.0      0.5      buy
#   2024-01-01 10:00:07  10:00:05     97345.23  97345.0      1.2      sell
# candles 샘플:
#   [10:00:00, 10:00:05, 10:00:10, 10:00:15, ...]
#
# price_levels 샘플:
#   [97330.0, 97335.0, 97340.0, 97345.0, 97350.0, ...]
candles = sorted(trades_df['candle_time'].unique())
price_levels = sorted(trades_df['price_level'].unique())

# ============================================================================
# Step 2: 히트맵용 매트릭스 생성 (가격 × 시간)
# ============================================================================
# delta_matrix 샘플 (shape: (가격 레벨 수, 캔들 수)):
#   [[ 0.7,  0.0, -0.3, ...],   <- 97340.0 가격 레벨의 Delta 값들
#    [ 0.0,  0.5,  0.0, ...],   <- 97345.0 가격 레벨의 Delta 값들
#    [-0.5,  0.0,  0.2, ...],   <- 97350.0 가격 레벨의 Delta 값들
#    ...]
#   각 행 = 가격 레벨, 각 열 = 캔들 시간
#
# buy_matrix, sell_matrix도 동일한 구조
delta_matrix = np.zeros((len(price_levels), len(candles)))
buy_matrix = np.zeros((len(price_levels), len(candles)))
sell_matrix = np.zeros((len(price_levels), len(candles)))

# ============================================================================
# Step 3: footprint_delta 데이터를 매트릭스로 변환
# ============================================================================
# footprint_delta 샘플:
#   {
#       (10:00:00, 97340.0): {'buy': 1.2, 'sell': 0.5, 'delta': 0.7, 'total': 1.7},
#       (10:00:05, 97345.0): {'buy': 0.8, 'sell': 0.3, 'delta': 0.5, 'total': 1.1},
#       ...
#   }
#
# 매트릭스 채우기:
#   delta_matrix[가격인덱스, 캔들인덱스] = footprint_delta[(캔들, 가격)]['delta']
for i, price in enumerate(price_levels):
    for j, candle in enumerate(candles):
        if (candle, price) in footprint_delta:
            data = footprint_delta[(candle, price)]
            delta_matrix[i, j] = data['delta']
            buy_matrix[i, j] = data['buy']
            sell_matrix[i, j] = data['sell']

# ============================================================================
# Step 4: 히트맵 축 레이블 생성
# ============================================================================
# x_labels 샘플:
#   ['10:00:00', '10:00:05', '10:00:10', '10:00:15', ...]
#
# y_labels 샘플:
#   ['$97,330', '$97,335', '$97,340', '$97,345', '$97,350', ...]
x_labels = [c.strftime('%H:%M:%S') for c in candles]
y_labels = [f"${p:,.0f}" for p in price_levels]

print(f"Delta Matrix shape: {delta_matrix.shape}")
print(f"Non-zero cells: {np.count_nonzero(delta_matrix)}/{delta_matrix.size}")

Delta Matrix shape: (52, 11)
Non-zero cells: 249/572


In [17]:
# Footprint Chart - Delta 히트맵
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Delta (매수 - 매도)', '매수 거래량', '매도 거래량'),
    vertical_spacing=0.08,
    row_heights=[0.4, 0.3, 0.3]
)

# 1. Delta 히트맵 (가장 중요)
fig.add_trace(
    go.Heatmap(
        z=delta_matrix,
        x=x_labels,
        y=y_labels,
        colorscale='RdYlGn',  # 빨강(매도)-노랑-초록(매수)
        zmid=0,  # 0을 중심으로
        colorbar=dict(title='Delta', x=1.02, len=0.3, y=0.85),
        hovertemplate='시간: %{x}<br>가격: %{y}<br>Delta: %{z:.4f}<extra></extra>',
        name='Delta'
    ),
    row=1, col=1
)

# 2. 매수 거래량
fig.add_trace(
    go.Heatmap(
        z=buy_matrix,
        x=x_labels,
        y=y_labels,
        colorscale='Greens',
        colorbar=dict(title='Buy Vol', x=1.02, len=0.25, y=0.48),
        hovertemplate='시간: %{x}<br>가격: %{y}<br>매수량: %{z:.4f}<extra></extra>',
        name='Buy Volume'
    ),
    row=2, col=1
)

# 3. 매도 거래량
fig.add_trace(
    go.Heatmap(
        z=sell_matrix,
        x=x_labels,
        y=y_labels,
        colorscale='Reds',
        colorbar=dict(title='Sell Vol', x=1.02, len=0.25, y=0.15),
        hovertemplate='시간: %{x}<br>가격: %{y}<br>매도량: %{z:.4f}<extra></extra>',
        name='Sell Volume'
    ),
    row=3, col=1
)

# 레이아웃 설정
fig.update_xaxes(title_text="시간", row=3, col=1)
fig.update_yaxes(title_text="가격", row=1, col=1)
fig.update_yaxes(title_text="가격", row=2, col=1)
fig.update_yaxes(title_text="가격", row=3, col=1)

fig.update_layout(
    title={
        'text': 'BTC/USDT Footprint Chart (발자국 차트)',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=1000,
    template='plotly_dark',
    showlegend=False
)

fig.show()

print("\n📊 Footprint Chart 해석:")
print("=" * 60)
print("✅ Delta 히트맵 (위):")
print("   - 초록색: 매수 압력 강함 (매수 > 매도)")
print("   - 빨간색: 매도 압력 강함 (매도 > 매수)")
print("   - 노란색: 균형 상태")
print("\n✅ 매수/매도 히트맵 (가운데/아래):")
print("   - 진한 색: 해당 가격/시간에서 거래량 많음")
print("   - 연한 색: 거래량 적음")
print("\n✅ 활용 방법:")
print("   - 특정 가격대에서 지속적으로 매수가 강하면 → 지지선")
print("   - 특정 가격대에서 지속적으로 매도가 강하면 → 저항선")
print("   - Delta가 급격히 변하는 구간 → 추세 전환 가능성")


📊 Footprint Chart 해석:
✅ Delta 히트맵 (위):
   - 초록색: 매수 압력 강함 (매수 > 매도)
   - 빨간색: 매도 압력 강함 (매도 > 매수)
   - 노란색: 균형 상태

✅ 매수/매도 히트맵 (가운데/아래):
   - 진한 색: 해당 가격/시간에서 거래량 많음
   - 연한 색: 거래량 적음

✅ 활용 방법:
   - 특정 가격대에서 지속적으로 매수가 강하면 → 지지선
   - 특정 가격대에서 지속적으로 매도가 강하면 → 저항선
   - Delta가 급격히 변하는 구간 → 추세 전환 가능성


In [18]:
### 고급 분석: Cumulative Delta

# 시간별 총 Delta 계산
delta_by_time = {}
for candle in candles:
    total_delta = 0
    for price in price_levels:
        if (candle, price) in footprint_delta:
            total_delta += footprint_delta[(candle, price)]['delta']
    delta_by_time[candle] = total_delta

# Cumulative Delta (누적 델타)
cumulative_delta = []
cumsum = 0
for candle in candles:
    cumsum += delta_by_time[candle]
    cumulative_delta.append(cumsum)

# 시각화
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('캔들별 Delta (Bar)', 'Cumulative Delta (누적)'),
    vertical_spacing=0.15,
    row_heights=[0.5, 0.5]
)

# 1. 캔들별 Delta
colors = ['green' if d > 0 else 'red' for d in delta_by_time.values()]
fig.add_trace(
    go.Bar(
        x=x_labels,
        y=list(delta_by_time.values()),
        marker_color=colors,
        name='Delta',
        hovertemplate='시간: %{x}<br>Delta: %{y:.4f}<extra></extra>'
    ),
    row=1, col=1
)

# 2. Cumulative Delta
fig.add_trace(
    go.Scatter(
        x=x_labels,
        y=cumulative_delta,
        mode='lines+markers',
        line=dict(color='cyan', width=3),
        marker=dict(size=6),
        name='Cumulative Delta',
        hovertemplate='시간: %{x}<br>누적 Delta: %{y:.4f}<extra></extra>'
    ),
    row=2, col=1
)

# 0선 추가
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_xaxes(title_text="시간", row=2, col=1)
fig.update_yaxes(title_text="Delta", row=1, col=1)
fig.update_yaxes(title_text="Cumulative Delta", row=2, col=1)

fig.update_layout(
    title='Delta 분석 - 시간별 매수/매도 압력',
    height=800,
    template='plotly_dark',
    showlegend=False
)

fig.show()

# 통계 출력
print("\n📈 Delta 분석 결과:")
print("=" * 60)
print(f"총 Delta: {cumulative_delta[-1]:.4f}")
if cumulative_delta[-1] > 0:
    print("→ 전체적으로 매수 압력이 우세했습니다.")
else:
    print("→ 전체적으로 매도 압력이 우세했습니다.")

positive_deltas = [d for d in delta_by_time.values() if d > 0]
negative_deltas = [d for d in delta_by_time.values() if d < 0]

print(f"\n양의 Delta 캔들: {len(positive_deltas)}개 (매수 우세)")
print(f"음의 Delta 캔들: {len(negative_deltas)}개 (매도 우세)")

if len(positive_deltas) > 0:
    print(f"평균 양의 Delta: {np.mean(positive_deltas):.4f}")
if len(negative_deltas) > 0:
    print(f"평균 음의 Delta: {np.mean(negative_deltas):.4f}")

print("\n✅ Cumulative Delta 해석:")
print("   - 상승하는 추세: 매수 압력이 지속적으로 강함")
print("   - 하락하는 추세: 매도 압력이 지속적으로 강함")
print("   - 가격은 오르는데 Cumulative Delta가 하락: 약세 다이버전스 (주의)")
print("   - 가격은 내리는데 Cumulative Delta가 상승: 강세 다이버전스 (반등 가능)")


📈 Delta 분석 결과:
총 Delta: -9.4068
→ 전체적으로 매도 압력이 우세했습니다.

양의 Delta 캔들: 4개 (매수 우세)
음의 Delta 캔들: 7개 (매도 우세)
평균 양의 Delta: 2.5919
평균 음의 Delta: -2.8249

✅ Cumulative Delta 해석:
   - 상승하는 추세: 매수 압력이 지속적으로 강함
   - 하락하는 추세: 매도 압력이 지속적으로 강함
   - 가격은 오르는데 Cumulative Delta가 하락: 약세 다이버전스 (주의)
   - 가격은 내리는데 Cumulative Delta가 상승: 강세 다이버전스 (반등 가능)


In [19]:
### 종합 차트: Footprint Chart (전문가 형식)

# ============================================================================
# Footprint Chart 형식: 각 캔들을 세로 막대로 표시
# - 왼쪽: 매도 거래량 (빨강)
# - 오른쪽: 매수 거래량 (녹색)
# - 검은색 강조: 해당 가격 레벨에서 가장 큰 거래량
# ============================================================================

# 캔들별 OHLC 계산
# candle_ohlc 샘플:
#   time        open      high      low       close
#   10:00:00    97340.0   97345.0   97335.0   97342.0
#   10:00:05    97342.0   97350.0   97340.0   97348.0
candle_ohlc = trades_df.groupby('candle_time').agg({
    'price': ['first', 'max', 'min', 'last']
}).reset_index()
candle_ohlc.columns = ['time', 'open', 'high', 'low', 'close']

# 차트 생성
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('가격 (OHLC)', 'Footprint Delta 히트맵', 'Cumulative Delta'),
    vertical_spacing=0.1,
    row_heights=[0.3, 0.5, 0.2],
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": False}]]
)

# 1. OHLC 캔들스틱
fig.add_trace(
    go.Candlestick(
        x=candle_ohlc['time'],
        open=candle_ohlc['open'],
        high=candle_ohlc['high'],
        low=candle_ohlc['low'],
        close=candle_ohlc['close'],
        name='Price',
        increasing_line_color='green',
        decreasing_line_color='red'
    ),
    row=1, col=1
)

# 2. Footprint Delta 히트맵
fig.add_trace(
    go.Heatmap(
        z=delta_matrix,
        x=x_labels,
        y=y_labels,
        colorscale='RdYlGn',
        zmid=0,
        colorbar=dict(title='Delta', x=1.02, len=0.4, y=0.5),
        hovertemplate='시간: %{x}<br>가격: %{y}<br>Delta: %{z:.4f}<extra></extra>',
        name='Footprint'
    ),
    row=2, col=1
)

# 3. Cumulative Delta
fig.add_trace(
    go.Scatter(
        x=x_labels,
        y=cumulative_delta,
        mode='lines+markers',
        line=dict(color='cyan', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 255, 255, 0.2)',
        name='Cumulative Delta'
    ),
    row=3, col=1
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)

# 레이아웃
fig.update_xaxes(title_text="시간", row=3, col=1)
fig.update_yaxes(title_text="가격 ($)", row=1, col=1)
fig.update_yaxes(title_text="가격 레벨", row=2, col=1)
fig.update_yaxes(title_text="누적 Delta", row=3, col=1)

fig.update_layout(
    title={
        'text': 'BTC/USDT - 가격 & Footprint 종합 차트',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    height=1200,
    template='plotly_dark',
    xaxis_rangeslider_visible=False,
    showlegend=False
)

fig.show()

print("\n🎯 종합 분석 방법:")
print("=" * 70)
print("1️⃣  가격 차트와 Footprint를 함께 보기")
print("   - 가격이 오르는데 Footprint에서 매수가 약하면 → 허약한 상승")
print("   - 가격이 내리는데 Footprint에서 매도가 약하면 → 허약한 하락")
print()
print("2️⃣  지지/저항 레벨 찾기")
print("   - Footprint에서 특정 가격대에 거래량이 집중 → 중요 레벨")
print("   - 그 레벨을 돌파할 때 Delta 방향 확인")
print()
print("3️⃣  다이버전스 찾기")
print("   - 가격 신고점 + Cumulative Delta 하락 → 약세 신호")
print("   - 가격 신저점 + Cumulative Delta 상승 → 강세 신호")
print()
print("4️⃣  주문 흐름의 지속성")
print("   - Cumulative Delta가 일관되게 한 방향 → 추세 강함")
print("   - Cumulative Delta가 횡보 → 추세 없음 (레인지)")
print("=" * 70)


🎯 종합 분석 방법:
1️⃣  가격 차트와 Footprint를 함께 보기
   - 가격이 오르는데 Footprint에서 매수가 약하면 → 허약한 상승
   - 가격이 내리는데 Footprint에서 매도가 약하면 → 허약한 하락

2️⃣  지지/저항 레벨 찾기
   - Footprint에서 특정 가격대에 거래량이 집중 → 중요 레벨
   - 그 레벨을 돌파할 때 Delta 방향 확인

3️⃣  다이버전스 찾기
   - 가격 신고점 + Cumulative Delta 하락 → 약세 신호
   - 가격 신저점 + Cumulative Delta 상승 → 강세 신호

4️⃣  주문 흐름의 지속성
   - Cumulative Delta가 일관되게 한 방향 → 추세 강함
   - Cumulative Delta가 횡보 → 추세 없음 (레인지)


In [25]:
### Footprint Chart - TradingView 스타일 (최적화 버전)

# ============================================================================
# TradingView 스타일 Footprint Chart (성능 최적화)
# - 중복 루프 제거: 한 번의 순회로 모든 계산 수행
# - 벡터화된 연산 사용
# - 불필요한 계산 제거
# ============================================================================

import time
start_time = time.time()

# 캔들별 OHLC 계산
candle_ohlc = trades_df.groupby('candle_time').agg({
    'price': ['first', 'max', 'min', 'last']
}).reset_index()
candle_ohlc.columns = ['time', 'open', 'high', 'low', 'close']
candle_ohlc_dict = {row['time']: row for _, row in candle_ohlc.iterrows()}

# ============================================================================
# 최적화: 한 번의 루프로 모든 데이터 준비
# ============================================================================
# 모든 셀 데이터를 리스트로 미리 준비 (Plotly 호출 최소화)
cell_data = []  # (candle_idx, price, sell_vol, buy_vol, is_poc, sell_color, buy_color)
candle_shapes = []  # (candle_idx, open, high, low, close, is_bullish)
annotation_data = []  # (x, y, text, color)

# 각 캔들에서 가장 큰 거래량 찾기 (POC 표시용) - 벡터화
max_volumes_per_candle = {}
candle_summary = {}

# 한 번의 순회로 모든 계산 수행
for j, candle in enumerate(candles):
    max_vol = 0
    total_delta = 0
    total_volume = 0
    
    # 해당 캔들의 모든 가격 레벨 처리
    for price in price_levels:
        key = (candle, price)
        if key in footprint_delta:
            data = footprint_delta[key]
            buy_vol = data['buy']
            sell_vol = data['sell']
            total_vol = data['total']
            
            if total_vol > 0:
                max_vol = max(max_vol, total_vol)
                total_delta += data['delta']
                total_volume += total_vol
    
    max_volumes_per_candle[candle] = max_vol
    candle_summary[candle] = {
        'delta': total_delta,
        'total': total_volume
    }

# 가격 레벨 간격 계산
price_step = price_levels[1] - price_levels[0] if len(price_levels) > 1 else 5.0
cell_height = price_step * 0.9

# 너비 설정
cell_width = 0.32
candle_body_width = 0.10
gap = 0.02

# 색상 정의
POC_COLOR = 'rgba(30, 30, 30, 0.95)'
CANDLE_UP = 'rgb(38, 166, 154)'
CANDLE_DOWN = 'rgb(239, 83, 80)'
CANDLE_BORDER = 'rgb(40, 40, 40)'

# 거래량 텍스트 포맷 함수
def format_vol(v):
    if v >= 1:
        return f'{v:.2f}'
    elif v >= 0.001:
        return f'{v:.3f}'
    else:
        return f'{v:.4f}'

# ============================================================================
# 데이터 준비 단계: 모든 셀과 캔들 정보를 미리 계산
# ============================================================================
for j, candle in enumerate(candles):
    candle_x = j
    max_vol = max_volumes_per_candle[candle]
    
    # 각 가격 레벨별로 셀 데이터 준비
    for price in price_levels:
        key = (candle, price)
        if key in footprint_delta:
            data = footprint_delta[key]
            buy_vol = data['buy']
            sell_vol = data['sell']
            total_vol = data['total']
            
            if total_vol == 0:
                continue
            
            # POC 여부
            is_poc = (total_vol == max_vol and total_vol > 0)
            
            # 색상 강도 계산
            if is_poc:
                sell_color = POC_COLOR
                buy_color = POC_COLOR
            else:
                sell_intensity = min(sell_vol / (max_vol + 0.001) * 0.5 + 0.4, 0.9)
                buy_intensity = min(buy_vol / (max_vol + 0.001) * 0.5 + 0.4, 0.9)
                sell_color = f'rgba(239, 83, 80, {sell_intensity})'
                buy_color = f'rgba(38, 166, 154, {buy_intensity})'
            
            # 셀 위치 계산
            sell_x1 = candle_x - candle_body_width/2 - gap
            sell_x0 = sell_x1 - cell_width
            buy_x0 = candle_x + candle_body_width/2 + gap
            buy_x1 = buy_x0 + cell_width
            
            # 셀 데이터 저장
            cell_data.append({
                'sell': (sell_x0, sell_x1, price, sell_color),
                'buy': (buy_x0, buy_x1, price, buy_color),
                'sell_vol': sell_vol,
                'buy_vol': buy_vol
            })
    
    # 캔들 정보 저장
    ohlc = candle_ohlc_dict.get(candle, None)
    if ohlc is not None:
        candle_shapes.append({
            'x': candle_x,
            'open': ohlc['open'],
            'high': ohlc['high'],
            'low': ohlc['low'],
            'close': ohlc['close']
        })
    
    # 하단 텍스트 저장
    summary = candle_summary[candle]
    delta_val = summary['delta']
    delta_color = 'rgb(38, 166, 154)' if delta_val >= 0 else 'rgb(239, 83, 80)'
    
    annotation_data.append({
        'type': 'delta',
        'x': candle_x,
        'y': min(price_levels) - price_step * 1.5,
        'text': f"Δ {delta_val:.3f}",
        'color': delta_color
    })
    annotation_data.append({
        'type': 'total',
        'x': candle_x,
        'y': min(price_levels) - price_step * 2.5,
        'text': f"{summary['total']:.3f}",
        'color': 'white'
    })

prep_time = time.time()
print(f"데이터 준비 시간: {prep_time - start_time:.3f}초")

# ============================================================================
# Plotly Figure 생성 및 렌더링
# ============================================================================
fig = go.Figure()

# 더미 trace 추가 (축 범위 설정용)
fig.add_trace(go.Scatter(
    x=[-0.5, len(candles) - 0.5],
    y=[min(price_levels) - price_step * 3.5, max(price_levels) + price_step],
    mode='markers',
    marker=dict(opacity=0),
    showlegend=False,
    hoverinfo='skip'
))

# 셀 그리기 (배치 처리)
for cell in cell_data:
    sell_x0, sell_x1, price, sell_color = cell['sell']
    buy_x0, buy_x1, price, buy_color = cell['buy']
    
    # 매도 셀
    fig.add_shape(
        type='rect',
        x0=sell_x0,
        x1=sell_x1,
        y0=price - cell_height/2,
        y1=price + cell_height/2,
        fillcolor=sell_color,
        line=dict(color='rgba(40,40,40,0.6)', width=0.5),
    )
    
    # 매수 셀
    fig.add_shape(
        type='rect',
        x0=buy_x0,
        x1=buy_x1,
        y0=price - cell_height/2,
        y1=price + cell_height/2,
        fillcolor=buy_color,
        line=dict(color='rgba(40,40,40,0.6)', width=0.5),
    )
    
    # 텍스트 (거래량이 있을 때만)
    if cell['sell_vol'] > 0:
        fig.add_annotation(
            x=(sell_x0 + sell_x1) / 2,
            y=price,
            text=format_vol(cell['sell_vol']),
            showarrow=False,
            font=dict(size=8, color='white'),
            xanchor='center',
            yanchor='middle'
        )
    
    if cell['buy_vol'] > 0:
        fig.add_annotation(
            x=(buy_x0 + buy_x1) / 2,
            y=price,
            text=format_vol(cell['buy_vol']),
            showarrow=False,
            font=dict(size=8, color='white'),
            xanchor='center',
            yanchor='middle'
        )

# 캔들스틱 그리기
for candle_info in candle_shapes:
    candle_x = candle_info['x']
    open_p = candle_info['open']
    high_p = candle_info['high']
    low_p = candle_info['low']
    close_p = candle_info['close']
    
    is_bullish = close_p >= open_p
    candle_fill = CANDLE_UP if is_bullish else CANDLE_DOWN
    
    body_top = max(open_p, close_p)
    body_bottom = min(open_p, close_p)
    
    if body_top - body_bottom < price_step * 0.1:
        body_top = (open_p + close_p) / 2 + price_step * 0.05
        body_bottom = (open_p + close_p) / 2 - price_step * 0.05
    
    # 캔들 바디
    fig.add_shape(
        type='rect',
        x0=candle_x - candle_body_width/2,
        x1=candle_x + candle_body_width/2,
        y0=body_bottom,
        y1=body_top,
        fillcolor=candle_fill,
        line=dict(color=CANDLE_BORDER, width=1),
        layer='above'
    )
    
    # 심지
    fig.add_shape(
        type='line',
        x0=candle_x,
        x1=candle_x,
        y0=body_top,
        y1=high_p,
        line=dict(color=candle_fill, width=2),
        layer='above'
    )
    fig.add_shape(
        type='line',
        x0=candle_x,
        x1=candle_x,
        y0=low_p,
        y1=body_bottom,
        line=dict(color=candle_fill, width=2),
        layer='above'
    )

# 하단 텍스트 추가
for ann in annotation_data:
    fig.add_annotation(
        x=ann['x'],
        y=ann['y'],
        text=ann['text'],
        showarrow=False,
        font=dict(size=9, color=ann['color']),
        xanchor='center'
    )

render_time = time.time()
print(f"렌더링 시간: {render_time - prep_time:.3f}초")
print(f"총 시간: {render_time - start_time:.3f}초")

# 레이아웃 설정
fig.update_layout(
    title={
        'text': 'BTC/USDT Footprint Chart',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16, 'color': 'white'}
    },
    xaxis=dict(
        title='',
        tickmode='array',
        tickvals=list(range(len(candles))),
        ticktext=[c.strftime('%H:%M:%S') for c in candles],
        showgrid=True,
        gridcolor='rgba(60, 60, 60, 0.4)',
        zeroline=False,
        tickfont=dict(color='rgba(200,200,200,0.8)', size=10)
    ),
    yaxis=dict(
        title='',
        showgrid=True,
        gridcolor='rgba(60, 60, 60, 0.4)',
        zeroline=False,
        tickfont=dict(color='rgba(200,200,200,0.8)', size=10),
        side='right'
    ),
    height=max(600, len(price_levels) * 18),
    template='plotly_dark',
    showlegend=False,
    plot_bgcolor='rgba(19, 23, 34, 1)',
    paper_bgcolor='rgba(19, 23, 34, 1)',
    margin=dict(l=20, r=60, t=50, b=50)
)

fig.show()

print("\n📊 Footprint Chart (TradingView Style - Optimized)")
print("=" * 60)
print("🔴 왼쪽 셀: 매도(Sell) 체결량 - 빨간/분홍 계열")
print("🟢 오른쪽 셀: 매수(Buy) 체결량 - 녹색/청록 계열")
print("📈 캔들: 상승=청록, 하락=빨강")
print("⬛ 검은 셀: POC (최다 거래량)")
print("📊 하단: Delta (Δ), Total Volume")
print("=" * 60)


데이터 준비 시간: 0.004초
렌더링 시간: 19.364초
총 시간: 19.368초



📊 Footprint Chart (TradingView Style - Optimized)
🔴 왼쪽 셀: 매도(Sell) 체결량 - 빨간/분홍 계열
🟢 오른쪽 셀: 매수(Buy) 체결량 - 녹색/청록 계열
📈 캔들: 상승=청록, 하락=빨강
⬛ 검은 셀: POC (최다 거래량)
📊 하단: Delta (Δ), Total Volume


---

## Footprint Chart 핵심 요약

### 배운 내용

1. **Footprint Chart란?**
   - 각 가격 레벨과 시간 구간에서 발생한 실제 거래량을 매수/매도로 구분
   - Orderbook(의도)과 달리 실제 체결된 거래(결과)를 보여줌
   - Delta = 매수량 - 매도량

2. **주요 지표**
   - **Delta**: 특정 시간/가격에서의 매수-매도 차이
   - **Cumulative Delta**: 누적 Delta (추세의 강도와 방향)
   - **Buy/Sell Volume**: 각각의 절대 거래량

3. **활용 방법**
   - **지지/저항 레벨 발견**: 거래량이 집중된 가격대
   - **추세 확인**: Cumulative Delta의 방향
   - **다이버전스 감지**: 가격과 Delta의 불일치
   - **체결 강도**: 특정 레벨 돌파 시 거래량 확인

4. **Orderbook과의 차이**
   ```
   Orderbook (호가창)     →  의도 (주문이 걸려있음)
   Footprint (발자국)     →  결과 (실제로 체결됨)
   ```

### 실전 트레이딩 팁

- ✅ **강한 상승**: 가격↑ + Cumulative Delta↑ (매수가 실제로 체결)
- ⚠️ **약한 상승**: 가격↑ + Cumulative Delta↓ (매도가 많이 체결, 위험)
- ✅ **강한 하락**: 가격↓ + Cumulative Delta↓ (매도가 실제로 체결)
- 🎯 **반등 가능**: 가격↓ + Cumulative Delta↑ (저점에서 매수 흡수)

### 다음 단계

실시간 대시보드에서 Orderbook과 Footprint를 동시에 모니터링하면 더욱 강력합니다!

```bash
# 대시보드 실행
uv run python -m intraday.dashboard
```

**참고**: Footprint Chart는 고급 분석 도구이며, 다른 지표들과 함께 사용할 때 가장 효과적입니다.